In [1]:
import pandas as pd

In [6]:
df = pd.read_parquet('../../hces-2023-24/data/sec-1-HCQ.parquet')  # , usecols=['sector', 'state_name', 'district_name'])
df = df[['sector', 'state_name', 'district_name', 'multiplier']]
df.head()

,sector,state_name,district_name,multiplier
hhid,,,,
46667201,rural,Jammu & Kashmir,Rajouri,16669
46667301,rural,Jammu & Kashmir,Rajouri,24840
46667302,rural,Jammu & Kashmir,Rajouri,24840
46667303,rural,Jammu & Kashmir,Rajouri,24840
46667304,rural,Jammu & Kashmir,Rajouri,24840


In [12]:
xdf = pd.read_parquet('../../hces-2023-24/data/sec-4-HCQ.parquet')
xdf = xdf[[
    'hh_size', 'employed_annual', 'max_income_from', 'hoh_religion', 'caste'
]]

In [14]:
df = pd.concat([df, xdf], axis=1, verify_integrity=True)

In [19]:
df.drop_duplicates(subset=[c for c in df if c != 'multiplier'])

,sector,state_name,district_name,multiplier,hh_size,employed_annual,max_income_from,hoh_religion,caste
hhid,,,,,,,,,
46667201,rural,Jammu & Kashmir,Rajouri,16669,5,yes,self-employment,Hinduism,others
46667301,rural,Jammu & Kashmir,Rajouri,24840,4,yes,regular wage/salary earning,Hinduism,others
46667302,rural,Jammu & Kashmir,Rajouri,24840,5,yes,self-employment,Islam,others
46667303,rural,Jammu & Kashmir,Rajouri,24840,5,yes,regular wage/salary earning,Hinduism,others
46667304,rural,Jammu & Kashmir,Rajouri,24840,3,no,None,Hinduism,others
...,...,...,...,...,...,...,...,...,...
41788202,rural,Ladakh,Leh,11676,2,yes,self-employment,Buddhism,scheduled tribe (ST)
41788301,rural,Ladakh,Leh,11503,4,no,None,Buddhism,scheduled tribe (ST)
41788302,rural,Ladakh,Leh,11503,4,yes,self-employment,Buddhism,scheduled tribe (ST)


In [31]:
df.drop(['multiplier', 'employed_annual'], axis=1, inplace=True)

In [33]:
df['hh_size']

hhid
46667201    5
46667301    4
46667302    5
46667303    5
46667304    3
           ..
43277314    4
43277315    5
43277316    4
43277317    5
43277318    2
Name: hh_size, Length: 261953, dtype: int64

In [38]:
df['family_size'] = pd.cut(df['hh_size'], [0, 1, 5, df['hh_size'].max()], labels=['single', 'nuclear', 'large'], right=True, include_lowest=True)

In [50]:
df[['hh_size', 'family_size']].sample(5)

,hh_size,family_size
hhid,,
45520305,6,large
44575301,5,nuclear
22388305,3,nuclear
25927201,3,nuclear
26690303,8,large


In [51]:
hdf = pd.read_parquet('../../hces-2023-24/data/sec-3-HCQ.parquet')
hdf.head()

rel_to_hoh  gender  age     marital_status  \
hhid     person_id                                                    
22300101 3          unmarried child    male   23      never married   
         2                   spouse  female   45  currently married   
         4          unmarried child  female   24      never married   
         1                     self    male   56  currently married   
22300201 2                   spouse  female   44  currently married   

                                  edu_level  edu_years used_internet  \
hhid     person_id                                                     
22300101 3                         graduate       18.0           yes   
         2                        secondary       10.0           yes   
         4          post graduate and above       20.0           yes   
         1                         graduate       15.0           yes   
22300201 2             upper primary/middle        8.0            no   

                    days_outside_home  meals_daily  meals_school  \
hhid     person_id                                                 
22300101 3                        0.0          3.0           NaN   
         2                        0.0          3.0           NaN   
         4                        0.0          3.0           NaN   
         1                        0.0          3.0           NaN   
22300201 2                        0.0          3.0           NaN   

                    meals_employer  meals_others  meals_payment  meals_home  \
hhid     person_id                                                            
22300101 3                     NaN           NaN            3.0        87.0   
         2                     NaN           3.0            NaN        87.0   
         4                     NaN           NaN            4.0        86.0   
         1                     NaN           NaN            3.0        87.0   
22300201 2                     NaN           NaN            NaN        90.0   

                    member_status  original_member  multiplier  
hhid     person_id                                              
22300101 3                      1              1.0       57436  
         2                      1              1.0       57436  
         4                      1              1.0       57436  
         1                      1              1.0       57436  
22300201 2                      1              1.0       27497

In [52]:
children = hdf['age'] <= 15
elderly = hdf['age'] >= 60

In [56]:
has_children = children.groupby(children.index.get_level_values(0)).any()

In [57]:
has_elderly = elderly.groupby(elderly.index.get_level_values(0)).any()

In [58]:
df['has_childred'] = has_children
df['has_elderly'] = has_elderly

In [62]:
df = df[['sector', 'state_name', 'district_name', 'max_income_from', 'hoh_religion', 'family_size', 'has_childred', 'has_elderly']]
df

,sector,state_name,district_name,max_income_from,hoh_religion,family_size,has_childred,has_elderly
hhid,,,,,,,,
46667201,rural,Jammu & Kashmir,Rajouri,self-employment,Hinduism,nuclear,False,False
46667301,rural,Jammu & Kashmir,Rajouri,regular wage/salary earning,Hinduism,nuclear,False,True
46667302,rural,Jammu & Kashmir,Rajouri,self-employment,Islam,nuclear,True,False
46667303,rural,Jammu & Kashmir,Rajouri,regular wage/salary earning,Hinduism,nuclear,True,True
46667304,rural,Jammu & Kashmir,Rajouri,None,Hinduism,nuclear,False,True
...,...,...,...,...,...,...,...,...
43277314,rural,Bihar,Samastipur,casual labour,Hinduism,nuclear,True,False
43277315,rural,Bihar,Samastipur,casual labour,Hinduism,nuclear,True,False
43277316,rural,Bihar,Samastipur,casual labour,Hinduism,nuclear,True,False


In [69]:
df['cohort_id'] = df.groupby(df.columns.tolist(), dropna=False).ngroup()

/var/folders/nz/p2j_7wnx56q0b61h97fxmmv40000gn/T/ipykernel_31656/1815201627.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['cohort_id'] = df.groupby(df.columns.tolist(), dropna=False).ngroup()
/var/folders/nz/p2j_7wnx56q0b61h97fxmmv40000gn/T/ipykernel_31656/1815201627.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cohort_id'] = df.groupby(df.columns.tolist(), dropna=False).ngroup()


In [79]:
df['cohort_id'].value_counts().tail(30)

cohort_id
28494    1
20916    1
28508    1
28497    1
28506    1
28498    1
28355    1
21554    1
21563    1
28342    1
20938    1
20933    1
28327    1
20936    1
28320    1
28333    1
28322    1
28319    1
28335    1
28308    1
28337    1
20919    1
28321    1
28336    1
20939    1
28318    1
28332    1
20917    1
20918    1
10009    1
Name: count, dtype: int64

In [84]:
df[df['cohort_id'] == 20918]

,sector,state_name,district_name,max_income_from,hoh_religion,family_size,has_childred,has_elderly,cohort_id
hhid,,,,,,,,,
47997314,rural,Uttar Pradesh,Kaushambi,casual labour,Islam,nuclear,True,True,20918
